## The Big Picture:

![IPython](venn_diagram.png "IPython")


#### The three core roles are referred as:
* Data Engineering (setting up the entire infrastructure)
* Data Analysis (providing statistical & mathematical analytics)
* Business Intelligence (or, in general: Domain specialist) (setting the context & objectives)

#### What all is involved in Data Engineering?
- **compute**: python, spark, hadoop 
- **database & storage**: git, SQL, NoSQL like MongoDB, Hbase, Postgres, sqlite
- **devops**: AWS, docker, mesos, repeatability
- **product**: UI, API, web, viz., story

#### Size issue choices:
- memory
- disk
- cluster
- doing computation where the data is!

#### Access issue choices:
- relational: SQL (pandas, sqlite, Postgres, Hbase, VoltDB)
- document oriented: MongoDB (stores file after file as JSON)
- key-value
- graph oriented (fb networks, twitter networks)

### Grammar of data:
<br/> Hadley Wickham formalized this as follows:
1. provide simple verbs for simple things (i.e. corresponding to common data manipulation tasks)
2. second idea is that backend does not matter (e.g. Pandas, sqlite, Julia, SQL database, spark etc. whatever it is)


#### Populating the Database

In [30]:
ourschema= """
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates"
    ("id" INTEGER PRIMARY KEY NOT NULL,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL);
CREATE TABLE "contributors"
    ("id" VARCHAR,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR,
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
    );
"""

#### SQLITE 
0. Postgres is recommended for production purposes, however sqlite is great for large databases which fit on-disk but which won't fit into memory.
1. Python implements a standard database API called DBAPI2, over most databases (sqlite is one of them). (There is an even high level API available called SQLAlchemy, but we won't use it here)

In [5]:
# Connect and get a DBAPI2 connection

from sqlite3 import dbapi2 as sq3
import os
PATHSTART="."

def get_db(dbfile):
    """Returns a connection to the database """
    sqlite_db= sq3.connect(os.path.join(PATHSTART,dbfile))
    return sqlite_db

In [6]:
# Drop tables if they exist & create them

def init_db(dbfile, schema):
    """Creates the database tables based on the schema"""
    db= get_db(dbfile)  ## get the connection to the database
    db.cursor().executescript(schema) ## create the tables using the schema
    db.commit() 
    return db

In [20]:
# Use pandas to read in the data
import pandas as pd

df_candi= pd.read_csv("./candidates.txt",sep="|")
df_cwci= pd.read_csv("./contributors_with_candidate_id.txt", sep="|")

#df_candi.head()
df_cwci.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [21]:
# but we want the 'id' in df_cwci to be autoincremented as mentioned in schema
del df_cwci['id']


In [25]:
df_cwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


---

#### Initializing the database

In [34]:
db= init_db("election_finance.db",ourschema)

#### Option 1: Populating with pandas

In [35]:
df_candi.to_sql("candidates",db, if_exists="append",index=False) #the index column of the dataframe means nothing for the database, so we put it as False
df_cwci.to_sql("contributors",db, if_exists="append",index=False)

#### Making queries

In [39]:
# selecting all the candidates and fetching them
sel= """
SELECT * FROM candidates;
"""
c= db.cursor().execute(sel)
c.fetchall()

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

In [63]:
# removing all the candidates
rem="""
DELETE FROM candidates;
"""
c= db.cursor().execute(rem)
db.commit()
c.fetchall()

[]

In [44]:
# Now that we have deleted the candidates table from our database, 
# its a good time to populate the database using Option 2

#### Option 2: Populating using SQL INSERT
Infact Option 1 won't work if the data that we are trying to enter is larger than the memory in the first place

In [78]:
ins= """
INSERT INTO candidates (id,first_name,last_name,middle_name,party) VALUES(?,?,?,?,?);
"""
slines=[]
with open("candidates.txt") as f:
    
    for l in f.readlines():
        slines.append(l.strip().split('|'))
        
    for line in slines[1:]: # we start from the 2nd line because the first line will be all column names
        theid, first_name, last_name, middle_name, party= line
        print(theid,first_name,last_name,middle_name,party)
        values_to_insert= (int(theid), first_name, last_name, middle_name, party)
        print(ins,values_to_insert,end='\n\n\n\n')
        db.cursor().execute(ins, values_to_insert)
        

33 Joseph Biden  D

INSERT INTO candidates (id,first_name,last_name,middle_name,party) VALUES(?,?,?,?,?);
 (33, 'Joseph', 'Biden', '', 'D')



36 Samuel Brownback  R

INSERT INTO candidates (id,first_name,last_name,middle_name,party) VALUES(?,?,?,?,?);
 (36, 'Samuel', 'Brownback', '', 'R')



34 Hillary Clinton R. D

INSERT INTO candidates (id,first_name,last_name,middle_name,party) VALUES(?,?,?,?,?);
 (34, 'Hillary', 'Clinton', 'R.', 'D')



39 Christopher Dodd J. D

INSERT INTO candidates (id,first_name,last_name,middle_name,party) VALUES(?,?,?,?,?);
 (39, 'Christopher', 'Dodd', 'J.', 'D')



26 John Edwards  D

INSERT INTO candidates (id,first_name,last_name,middle_name,party) VALUES(?,?,?,?,?);
 (26, 'John', 'Edwards', '', 'D')



22 Rudolph Giuliani  R

INSERT INTO candidates (id,first_name,last_name,middle_name,party) VALUES(?,?,?,?,?);
 (22, 'Rudolph', 'Giuliani', '', 'R')



24 Mike Gravel  D

INSERT INTO candidates (id,first_name,last_name,middle_name,party) VALUES(?,?,?,?,?);

#### Making some queries

In [79]:
# selecting and fetching all rows
def make_query(sel):
    c=db.cursor().execute(sel)
    return c.fetchall()

In [80]:
make_query("SELECT * FROM candidates")

[(16, 'Mike', 'Huckabee', '', 'R'),
 (20, 'Barack', 'Obama', '', 'D'),
 (22, 'Rudolph', 'Giuliani', '', 'R'),
 (24, 'Mike', 'Gravel', '', 'D'),
 (26, 'John', 'Edwards', '', 'D'),
 (29, 'Bill', 'Richardson', '', 'D'),
 (30, 'Duncan', 'Hunter', '', 'R'),
 (31, 'Dennis', 'Kucinich', '', 'D'),
 (32, 'Ron', 'Paul', '', 'R'),
 (33, 'Joseph', 'Biden', '', 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', '', 'R'),
 (36, 'Samuel', 'Brownback', '', 'R'),
 (37, 'John', 'McCain', '', 'R'),
 (38, 'Tom', 'Tancredo', '', 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

### Making many more Queries/ Verbs

#### Single table Verbs
See https://gist.github.com/TomAugspurger/6e052140eaa5fdb6e8c0/ which has a comparison of R/dplyr and pandas. I stole and modified this table from there:

``dplyr`` has a small set of nicely defined verbs. I've listed their closest pandas verbs.


<table>
  <tr>
    <th><b>VERB</b></th>
    <th><b>dplyr</b></th>
    <th><b>pandas</b></th>
    <th><b>SQL</b></th>
  </tr>
  <tr>
    <td>QUERY/SELECTION</td>
    <td>filter() (and slice())</td>
    <td>query() (and loc[], iloc[])</td>
    <td>SELECT WHERE</td>
  </tr>
  <tr>
    <td>SORT</td>
    <td>arrange()</td>
    <td>sort()</td>
    <td>ORDER BY</td>
  </tr>
  <tr>
    <td>SELECT-COLUMNS/PROJECTION</td>
    <td>select() (and rename())</td>
    <td>[](__getitem__) (and rename())</td>
    <td>SELECT COLUMN</td>
  </tr>
  <tr>
    <td>SELECT-DISTINCT</td>
    <td>distinct()</td>
    <td>unique(),drop_duplicates()</td>
    <td>SELECT DISTINCT COLUMN</td>
  </tr>
  <tr>
    <td>ASSIGN</td>
    <td>mutate() (and transmute())</td>
    <td>assign</td>
    <td>ALTER/UPDATE</td>
  </tr>
  <tr>
    <td>AGGREGATE</td>
    <td>summarise()</td>
    <td>describe(), mean(), max()</td>
    <td>None, AVG(),MAX()</td>
  </tr>
  <tr>
    <td>SAMPLE</td>
    <td>sample_n() and sample_frac()</td>
    <td>sample()</td>
    <td>implementation dep, use RAND()</td>
  </tr>
  <tr>
    <td>GROUP-AGG</td>
    <td>group_by/summarize</td>
    <td>groupby/agg, count, mean</td>
    <td>GROUP BY</td>
  </tr>
  <tr>
    <td>DELETE</td>
    <td>?</td>
    <td>drop/masking</td>
    <td>DELETE/WHERE</td>
  </tr>
</table>

#### Verbs involving multiple tables